In [5]:
%run ../yp_utils.py

# Initial setup

In [6]:
paper_pmid = 19054128
paper_name = 'yoshikawa_shimizu_2009' 

In [7]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [8]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
original_data = pd.read_excel('raw_data/FYR_456_sm_tableS1.xlsx', sheet_name='data', skiprows=1)

In [13]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4729 x 14


In [14]:
original_data.head()

,Name,Name.1,specific growth rate at 1st experiment (1/h),specific growth rate at 2nd experiment (1/h),"growth behavior (0: sensitive, 1: no-change, 2: tolerant)",specific growth rate at 1st experiment (1/h) .1,specific growth rate at 2nd experiment (1/h) .1,"growth behavior (0: sensitive, 1: no-change, 2: tolerant).1",specific growth rate at 1st experiment (1/h) .2,specific growth rate at 2nd experiment (1/h) .2,"growth behavior (0: sensitive, 1: no-change, 2: tolerant).2",specific growth rate at 1st experiment (1/h) .3,specific growth rate at 2nd experiment (1/h) .3,"growth behavior (0: sensitive, 1: no-change, 2: tolerant).3"
0,YAL068C,YAL068C,0.4549,0.4505,1,0.3045,0.3187,1,0.1795,0.1839,1,0.2066,0.2048,1
1,YAL067C,SEO1,0.4499,0.4423,1,0.2923,0.3006,1,0.1812,0.1855,1,0.2093,0.1988,1
2,YAL066W,YAL066W,0.4558,0.4759,1,0.3023,0.3112,1,0.1649,0.1565,1,0.2000,0.1896,1
3,YAL065C,YAL065C,0.4411,0.4447,1,0.3028,0.3138,1,0.1759,0.1736,1,0.1565,0.1470,0
4,YAL062W,GDH3,0.4548,0.4803,1,0.3194,0.3160,2,0.1652,0.1664,1,0.1911,0.1838,1


In [15]:
original_data['orf'] = original_data['Name'].astype(str)

In [16]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [17]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [18]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Name, Name.1, specific growth rate at 1st experiment (1/h) , specific growth rate at 2nd experiment (1/h) , growth behavior (0: sensitive, 1: no-change, 2: tolerant), specific growth rate at 1st experiment (1/h) .1, specific growth rate at 2nd experiment (1/h) .1, growth behavior (0: sensitive, 1: no-change, 2: tolerant).1, specific growth rate at 1st experiment (1/h) .2, specific growth rate at 2nd experiment (1/h) .2, growth behavior (0: sensitive, 1: no-change, 2: tolerant).2, specific growth rate at 1st experiment (1/h) .3, specific growth rate at 2nd experiment (1/h) .3, growth behavior (0: sensitive, 1: no-change, 2: tolerant).3, orf]
Index: []


In [19]:
original_data.set_index('orf', inplace=True)

In [21]:
data_cols = [c for c in original_data.columns if 'specific growth rate' in c]

In [22]:
original_data = original_data[data_cols].copy()

In [23]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [24]:
original_data = original_data.groupby(original_data.index).mean()

In [25]:
original_data.shape

(4705, 8)

In [27]:
dataset_ids = [100, 100, 523, 523, 4, 4, 5, 5]
original_data.columns = dataset_ids

In [28]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

In [29]:
original_data.shape

(4705, 4)

In [31]:
# Normalize by the untreated sample
original_data = original_data.div(original_data[100], axis=0)
original_data.head()

,4,5,100,523
orf,,,,
YAL002W,0.369395,0.433857,1.0,0.682175
YAL004W,0.381866,0.484486,1.0,0.716042
YAL005C,0.358770,0.433311,1.0,0.689781
YAL007C,0.371400,0.355228,1.0,0.685645
YAL008W,0.375338,0.443483,1.0,0.674310


In [32]:
original_data.drop(columns=[100], inplace=True)

# Prepare the final dataset

In [33]:
data = original_data.copy()

In [34]:
dataset_ids = original_data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [35]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [36]:
data.head()

dataset_id,4,5,523
data_type,value,value,value
orf,,,
YAL002W,0.369395,0.433857,0.682175
YAL004W,0.381866,0.484486,0.716042
YAL005C,0.358770,0.433311,0.689781
YAL007C,0.371400,0.355228,0.685645
YAL008W,0.375338,0.443483,0.674310


## Subset to the genes currently in SGD

In [37]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [38]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4,5,523
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,0.369395,0.433857,0.682175
1863,YAL004W,0.381866,0.484486,0.716042
4,YAL005C,0.358770,0.433311,0.689781
5,YAL007C,0.371400,0.355228,0.685645
6,YAL008W,0.375338,0.443483,0.674310


# Normalize

In [39]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [40]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [41]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4,5,523,4,5,523
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,0.369395,0.433857,0.682175,-0.356622,-0.273392,0.294598
1863,YAL004W,0.381866,0.484486,0.716042,-0.133611,0.660291,1.104287
4,YAL005C,0.358770,0.433311,0.689781,-0.546602,-0.283448,0.476443
5,YAL007C,0.371400,0.355228,0.685645,-0.320761,-1.723406,0.377552
6,YAL008W,0.375338,0.443483,0.674310,-0.250345,-0.095867,0.106577


# Print out

In [42]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [43]:
from IO.save_data_to_db3 import *

In [44]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 19054128...
Inserting the new data...


100%|██████████| 3/3 [00:20<00:00,  6.82s/it]

Updating the data_modified_on field...
